<a href="https://colab.research.google.com/github/AnshumanJain101/Recommendation-System/blob/main/Matrix_Factorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import sys
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from scipy import sparse

df = pd.read_csv('ratings.csv')




df_utility = df.pivot(index = 'userId',columns='movieId',values = 'rating').fillna(0)
print(df_utility.head())

#creating numpy array from pandas dataframe cz it provides better computn

ratings = np.array(df_utility)
# ratings numpy utility matrix

print(ratings)



  # nonzeroarr = ratings[2,:].nonzero()[0]
  # print(nonzeroarr)
  # nonzerolen = len(nonzeroarr)
  # print(nonzerolen)
  # test_rating_indices = np.random.choice(nonzeroarr,size=int(nonzerolen*0.10),replace=False)
  # print(test_rating_indices) 
  # the cocept of splitting is ki 10 maan lo ratings maine train mai 0 kr di aur unke corresponding test mai daal di  
  def split_testtrain(movie_ratings,value_of_split):
    testing_data = np.zeros(movie_ratings.shape)
        
    training_data = movie_ratings.copy()
    for i in range(movie_ratings.shape[0]):
      nonzeroarr = movie_ratings[i,:].nonzero()[0]
      # print(nonzeroarr[0])
      nonzerolen = len(nonzeroarr)
      # print(nonzerolen)
      indices_test = np.random.choice(nonzeroarr,size=int(nonzerolen*value_of_split),replace=False)
      #training_rating_ind = np.random.choice(nonzeroarr,size=int(nonzerolen*(1-value_of_split)),replace=False)
      training_data[i,indices_test]=0
      testing_data[i,indices_test] = movie_ratings[i,indices_test]

    assert(np.all((training_data*testing_data)==0))

  # so after assert now we can return train and test data

    return training_data,testing_data


# Lets call and check train test separation


train,test = split_testtrain(ratings,0.3)



def matrix_factorization_main(training_data,user_latent_features,movies_latent_features,K,steps,l_rate,reg_parameter):
    movies_latent_features = movies_latent_features.T
    for step in range(steps):
        print step
        #for each user
        for i in range(training_data.shape[0]):
            #for each item
            for j in range(training_data.shape[1]):
                if training_data[i][j] > 0 :

                    #calculate the error of the element
                    eij = training_data[i][j] - np.dot(user_latent_features[i,:],movies_latent_features[:,j])
                    
                    #print eij
                    #compute the gradient from the error
                    for k in range(K):
                        user_latent_features[i][k] = user_latent_features[i][k] + l_rate * ( 2 * eij * movies_latent_features[k][j] - (reg_parameter * user_latent_features[i][k]))
                        movies_latent_features[k][j] = movies_latent_features[k][j] + l_rate * (2 * eij * user_latent_features[i][k] - (reg_parameter * movies_latent_features[k][j]))

        #compute total error
        error = 0
        #for each user
        for i in range(training_data.shape[0]):
            #for each item
            for j in range(training_data.shape[1]):
                if training_data[i][j] > 0:
                    error += np.power(training_data[i][j] - np.dot(user_latent_features[i,:],movies_latent_features[:,j]),2)
        if error < 0.001:
            break
    return user_latent_features, movies_latent_features.T



def get_rmse(pred, actual):
    # Ignore zero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

def get_mae(pred, actual):
    # Ignore zero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_absolute_error(pred, actual)


training_data = train
len_traing_data = len(training_data)
no_of_movies = len(training_data[0])

#latent features chosen are 2 for simplicity
K = 2
P = numpy.random.rand(len_training_data,K)
Q = numpy.random.rand(no_of_movies,K)
nP, nQ = matrix_factorization_main(training_data, P, Q, K)
predict = numpy.dot(nP, nQ.T)

error = (predict,test)
print(error)

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931
Total Users : 610
Total Movie : 9724
movieId  1       2       3       4       ...  193583  193585  193587  193609
userId                                   ...                                
1           4.0     0.0     4.0     0.0  ...     0.0     0.0     0.0     0.0
2           0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
3           0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
4           0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
5           4.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0

[5 rows x 9724 columns]
Data splitted into Train,Test


KeyboardInterrupt: ignored